# Extracción de datos desde APIs con Python


## Conceptos básicos de APIs REST y consumo de APIs en Python

### ¿Qué son las API?

- Son un conjuntos de reglas y protocolos que permiten a diferentes aplicaciones comunicarse entre sí. 
- Se utilizan para acceder a datos de servicios web, aplicaciones en la nube, redes sociales, entre otros. 
- Proporcionan una forma estructurada y segura de obtener datos de sistemas externos.

![API](../Images/API.png)

## Extracción de datos JSON desde APIs públicas y privadas

![API_Spotify](../Images/API_Spotify.png)

In [18]:
import pandas as pd
import os
import json
import requests
import re
import base64
from requests.auth import HTTPBasicAuth

In [19]:
path = os.getcwd()
path_credenciales = os.path.join(path, 'secretos', 'credentials_spoty.json')
with open(path_credenciales) as f:
    credenciales = json.load(f)
    client_id = credenciales['client_id']
    client_secret = credenciales['client_secret']

data = {
    'grant_type': 'client_credentials',
    'client_id': client_id,
    'client_secret': client_secret
}

token_url = 'https://accounts.spotify.com/api/token'
response = requests.post(token_url, data=data)

if response.status_code == 200:
    token = response.json()['access_token']
    print('Token válido')
else:
    print('Error en la obtención del token')
    print(response.text)

Token válido


In [20]:
id_track = '2plbrEY59IikOBgBGLjaoe'
url = f'https://api.spotify.com/v1/audio-features/{id_track}'
headers = {
    'Authorization': f'Bearer {token}'
}
response = requests.get(url, headers=headers)
response.json()

{'error': {'status': 403}}

Consumo de la API de [mockaroo](https://www.mockaroo.com/), que es una herramienta que permite generar datos de prueba en diferentes formatos, como JSON, CSV, SQL y Excel.

In [14]:
import requests
headers = {'X-Api-Key': 'd67d54e0'}
response = requests.get('https://my.api.mockaroo.com/mkt_data_cf.json', headers=headers)

In [15]:
response.json()

{'error': 'Invalid API Key'}

### Pandas JSON

In [16]:
import pandas as pd
from io import StringIO

In [17]:
pd.read_json(StringIO(response.text))

ValueError: If using all scalar values, you must pass an index

In [11]:
path_data = os.path.join(os.getcwd(), 'data')

ejemplo1 = os.path.join(path_data, 'ejemplo1.json')
ejemplo2 = os.path.join(path_data, 'ejemplo2.json')
ejemplo3 = os.path.join(path_data, 'ejemplo3.json')

#### Ejemplo 1

In [12]:
pd.read_json(ejemplo1)

,nombre,productos
0,Tienda de Electrónica,"{'nombre_prod': 'Laptop', 'precio': 800, 'marc..."
1,Tienda de Electrónica,"{'nombre_prod': 'Teléfono', 'precio': 500, 'ma..."


#### json_normalize

**¿Qué hace `pd.json_normalize`?**

En esencia, esta función de Pandas toma datos JSON potencialmente complejos y anidados, y los transforma en un DataFrame plano y estructurado, que es mucho más fácil de manipular y analizar.

**Desglose de los parámetros:**

* `data`: Este es el JSON que contiene tus datos.

* `record_path='productos'`: Aquí le estás diciendo a Pandas: "Dentro de mi diccionario `data`, busca una lista llamada 'productos'. Cada elemento dentro de esa lista representa una fila en mi DataFrame final".

* `meta=['nombre']`:  Con esto indicas: "Además de los datos dentro de 'productos', quiero incluir una columna adicional llamada 'nombre' en mi DataFrame. El valor de esta columna será el valor asociado a la clave 'nombre' en el nivel superior de mi diccionario `data`".  En nuestro ejemplo, esto significa que cada fila tendrá el nombre de la tienda ("Tienda de Electrónica") junto con los detalles del producto.


In [13]:
data = json.load(open(ejemplo1))
pd.json_normalize(
    data,
    record_path='productos',
    meta=['nombre']
)

,nombre_prod,precio,marca,nombre
0,Laptop,800,Marca A,Tienda de Electrónica
1,Teléfono,500,Marca B,Tienda de Electrónica


#### Ejemplo 2

In [14]:
pd.read_json(ejemplo2)

,usuario
compras,"[{'producto': 'Libro', 'precio': 20, 'fecha': ..."
direcciones,"[{'calle': 'Calle Principal 123', 'ciudad': 'C..."
edad,30
nombre,Juan


In [15]:
data2 = json.load(open(ejemplo2))
pd.json_normalize(
    data2
)

,usuario.nombre,usuario.edad,usuario.direcciones,usuario.compras
0,Juan,30,"[{'calle': 'Calle Principal 123', 'ciudad': 'C...","[{'producto': 'Libro', 'precio': 20, 'fecha': ..."


In [18]:
pd.json_normalize(
    data2['usuario'],
    record_path=['direcciones'],
    meta=['nombre', 'edad',]
)

,calle,ciudad,id,nombre,edad
0,Calle Principal 123,Ciudad A,1,Juan,30
1,Avenida Secundaria 456,Ciudad B,2,Juan,30


In [19]:
pd.json_normalize(
        data2['usuario'],
    record_path=['compras'],
    meta=['nombre', 'edad',]
)

,producto,precio,fecha,id,nombre,edad
0,Libro,20,2020-01-01,1,Juan,30
1,Película,15,2020-01-02,2,Juan,30


#### Ejemplo 3

In [20]:
pd.read_json(ejemplo3)

,escuela
grados,"[{'nombre': 'Primer Grado', 'alumnos': [{'nomb..."
nombre,Escuela Primaria


In [21]:
data3 = json.load(open(ejemplo3))
pd.json_normalize(
    data3
)

,escuela.nombre,escuela.grados
0,Escuela Primaria,"[{'nombre': 'Primer Grado', 'alumnos': [{'nomb..."


In [22]:
pd.json_normalize(
    data3['escuela']['grados'],
    record_path='alumnos',
    meta=['nombre'],
    meta_prefix='grado_'
)

,nombre,edad,grado_nombre
0,Ana,6,Primer Grado
1,Pedro,7,Primer Grado
2,María,7,Segundo Grado
3,Luis,8,Segundo Grado
